# Machine Learning
The code below creates a random forest regressor, which is the main model I discuss in the report. This notebook also contains the material to evaluate the model and answer all of the questions in the second part of the task. These answers are detailed in section 2 of the report. Before runing, make sure that the covariates data file is saved as "data/tcga.csv".

### Imports

In [4]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
import forestci as fci


### Global variables

In [3]:
CWD = os.path.abspath(os.path.join(os.path.abspath(''), ".."))
ROOT_DIR = os.path.join(CWD, "../")
DATA_DIR = os.path.join(ROOT_DIR, "data")


### Load the data
This cell can be skipped along with the next cell and the prep-processed datafile loaded instead, but is included here to show how the pre-processing took place.

In [18]:
data_file = "tcga.csv"
df = pd.read_csv(os.path.join(DATA_DIR, data_file))

### Task 2.1 - Pre-processing
This cell pre-processes the code and saves a new version of the data file, so that this step does not need to be repeated every session.
If the above cell is skipped to avoid loading the data twice, this cell must also be skipped.

In [19]:
""" NB: There are no duplicate rows."""
duplicate_rows = "yes" if len(df) > len(df.drop_duplicates()) else "No"
print(f"Are there duplicate rows? {duplicate_rows}")
"""NB: Are columns are float64 except treatment which is int64."""
show_dtypes = False
if show_dtypes:
    print(df.dtypes)
""" NB: There are no columns that contain a single value
    (no zero-variance features)."""
zero_variance_features = "yes" if len(df.columns) != len(df.loc[:, (df != df.iloc[0]).any()].columns) else "No"
print(f"Are there any zero-variance features? {zero_variance_features}")
""" NB: There are columns that have near-zero variance. These
    will not be removed in the first instance, but it is
    worth baring them in mind. If we set our threshold for
    "near-zero" variance to containing less than 1% unique values,
    they are: gene_332, gene_573, gene_836, gene_837, gene_838, gene_839,
    gene_841, gene_871, gene_891, gene_1530, gene_1534, gene_1540,
    gene_1976, gene_1977, gene_1978, gene_2106 and gene_3063."""
near_zero_v_columns = [col for col in df.columns if len(df[col].unique()) <= len(df[col]) * 0.01 and col != "treatment"]
print(f"The near-zero-variance features: {near_zero_v_columns}")

""" NB: Columns do contain missing values and will be imputed with the median
    value. This does not factor the covariance between features, but there
    are too many missing values to impute with linear regression."""
missing_value_count = df.isnull().sum().sum()
print(f"How many missing values are there? {missing_value_count}")

# Impute the missing values
df.replace(np.NaN, df.median(), inplace=True)

# Save pre-processed dataset to file
preprocessed_outfile = os.path.join(DATA_DIR, "tcga_preprocessed.csv")
df.to_csv(preprocessed_outfile, index=False)


Are there duplicate rows? No
Are there any zero-variance features? No
The near-zero-variance features: ['gene_332', 'gene_573', 'gene_836', 'gene_837', 'gene_838', 'gene_839', 'gene_841', 'gene_871', 'gene_891', 'gene_1530', 'gene_1534', 'gene_1540', 'gene_1976', 'gene_1977', 'gene_1978', 'gene_2106', 'gene_3063']
Are there missing values? 3862128


### Load the pre-processed data file

In [ ]:
preprocessed_outfile = os.path.join(DATA_DIR, "tcga_preprocessed.csv")
df = pd.read_csv(os.path.join(DATA_DIR, preprocessed_outfile))
feature_columns = [c for c in df.columns if "gene_" in c or c == "treatment"]
feature_df = df[feature_columns]
outcome = df["outcome"]

### Define the train/test split
The near-zero-variance columns are also stored here for ease, if we want to remove them from the training set.

In [ ]:
near_zero_v_columns = [
    "gene_332",
    "gene_573",
    "gene_836",
    "gene_837",
    "gene_838",
    "gene_839",
    "gene_841",
    "gene_871",
    "gene_891",
    "gene_1530",
    "gene_1534",
    "gene_1540",
    "gene_1976",
    "gene_1977",
    "gene_1978",
    "gene_2106",
    "gene_3063",
]
v_columns = [col for col in feature_columns if col not in near_zero_v_columns]
feature_df_no_near_zero_variance = feature_df[v_columns]

X_train, X_test, y_train, y_test = train_test_split(
    feature_df, outcome, test_size=0.25, random_state=5
)

### Task 2.2 - Define the model and fit

In [ ]:
regressor = RandomForestRegressor(
    n_estimators=200,
    min_samples_leaf=1,
    max_samples=0.7,
    max_features="auto",
    max_depth=4,
    random_state=5,
)
regressor.fit(X_train, y_train)

### Predict values
Use the model to predict both the train and test sets and store these values in two dataframes. The predicted test values are used to evaluate the model. Comparing the error in the predicted values for each set shows the extent of over-fitting of the model.  

In [ ]:
y_pred = regressor.predict(X_test)
y_pred_train = regressor.predict(X_train)

pred_df = pd.DataFrame(
    data={
        "outcome": y_test,
        "RF_Prediction": y_pred,
        "error": abs(y_test - y_pred),
        "squared_error": (y_test - y_pred) ** 2,
    }
)

pred_df_train = pd.DataFrame(
    data={
        "outcome": y_train,
        "RF_Prediction": y_pred_train,
        "error": y_train - y_pred_train,
        "squared_error": (y_train - y_pred_train) ** 2,
    }
)
display(pred_df)

### Task 2.3 and 2.5 - Model Evaluation
Print the Root Mean Squared Error (RMSE) and $R^{2}$ for each set of predicted values

In [ ]:
print(
    "Training RMSE: ",
    metrics.mean_squared_error(y_train, y_pred_train, squared=False),
)
print(
    "Test RMSE: ",
    metrics.mean_squared_error(y_test, y_pred, squared=False),
)

print("Training R^2: ", metrics.r2_score(y_train, y_pred_train))
print("Test R^2: ", metrics.r2_score(y_test, y_pred))

### Task 2.5 - Uncertainty - not working
The following "commented out" cell was intended to calculate and plot the uncertainties in any given predicted value, the below code does not work as the random_forest_error currently fails to produce error values when calibrated. When uncalibrated it produces invalid values. If given more time, I would like to explore a possible work-around for this, as discussed in these open issues ([#72](https://github.com/scikit-learn-contrib/forest-confidence-interval/issues/72), [#78](https://github.com/scikit-learn-contrib/forest-confidence-interval/issues/78), and [#83](https://github.com/scikit-learn-contrib/forest-confidence-interval/issues/83)) in github.

In [ ]:
# # Calculate the variance
# # Plot predicted outcome without error bars
# plt.scatter(pred_df["outcome"], pred_df["RF_Prediction"])
# plt.xlabel("Reported outcome")
# plt.ylabel("Predicted outcome")
# plt.show()

# # Calculate the variance - More work required to get fci.random_forest_error working
# outcome_forrest_error = fci.random_forest_error(regressor, X_train, X_test, calibrate=True)

# # Plot error bars for predicted outcome using unbiased variance
# plt.errorbar(pred_df["outcome"], pred_df["RF_Prediction"], yerr=np.sqrt(outcome_forrest_error), fmt="o")
# plt.xlabel("Reported outcome")
# plt.ylabel("Predicted outcome")
# plt.show()

###  Task 2.4 - Model errors

In [ ]:
print("Here are the rows in the top 1% for squared error:")
display(pred_df.nlargest(len(pred_df) // 100, columns="squared_error", keep="all"))

### Task 2.6 - Feature importance

In [ ]:
drop_treatment = False

feature_importance_df = pd.DataFrame(
    data={
        "Feature": [f"gene_{i}" for i in range(4000)] + ["treatment"],
        "importance": regressor.feature_importances_,
    }
)
if drop_treatment:
    feature_importance_df = feature_importance_df.drop(
        feature_importance_df.tail(1).index, inplace=True
    )
    n = 20
else:
    n = 21
    
feature_importance_df.plot()
display(feature_importance_df.nlargest(n, columns="importance", keep="all"))

### Task 2.2 and 2.3 further - Grid searches and analysis
The below code sets up several cross-validated grid searches for hyper-parameter tuning. It then outputs the results to a file and plots the performance of the model as different parameters are varied.

Set up and fit grid search

In [ ]:
regressor = RandomForestRegressor(random_state=5)
params = {
    "n_estimators": [100, 200],
    "max_features": ["auto", "sqrt", "log2"],
    "max_depth": [4, 6, 8],
    "criterion": ["squared_error", "absolute_error"],
}
CV_regressor = GridSearchCV(
    estimator=regressor, param_grid=params, n_jobs=-1, cv=3, verbose=2
)
CV_regressor.fit(X_train, y_train)

Output and display results

In [ ]:
cv_results = pd.DataFrame(data=CV_regressor.cv_results_)
cv_outfile = os.path.join(ROOT_DIR, "output/random-forest_gridsearch_results.csv")
cv_results.to_csv(cv_outfile, index=False)
display(pd.DataFrame(data=CV_regressor.cv_results_))

Plot bar chart for results

In [ ]:
for idx, group in cv_results.groupby(
    by=["param_max_depth", "param_criterion", "param_n_estimators"]
):
    fig, ax = plt.subplots()
    max_features = ["auto", "sqrt", "log2"]
    x_pos = range(len(max_features))
    auto_mean = group.loc[group["param_max_features"] == "auto"][
        "mean_test_score"
    ].values[0]
    sqrt_mean = group.loc[group["param_max_features"] == "sqrt"][
        "mean_test_score"
    ].values[0]
    log2_mean = group.loc[group["param_max_features"] == "log2"][
        "mean_test_score"
    ].values[0]
    auto_std = group.loc[group["param_max_features"] == "auto"][
        "std_test_score"
    ].values[0]
    sqrt_std = group.loc[group["param_max_features"] == "sqrt"][
        "std_test_score"
    ].values[0]
    log2_std = group.loc[group["param_max_features"] == "log2"][
        "std_test_score"
    ].values[0]

    test_scores = [auto_mean, sqrt_mean, log2_mean]
    error = [auto_std, sqrt_std, log2_std]
    ax.bar(
        x_pos,
        test_scores,
        yerr=error,
        align="center",
        alpha=0.5,
        ecolor="black",
        capsize=10,
    )
    ax.set_ylabel("Test score")
    ax.set_xlabel("max_features")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(max_features)
    ax.set_title(
        f"max_features comparison - max_depth={idx[0]}, criterion={idx[1]}, n_estimators={idx[2]}"
    )
    ax.yaxis.grid(True)

    # Save the figure and show
    plt.tight_layout()
    max_features_comparison_plot_file = os.path.join(
        ROOT_DIR,
        f"output/machine_learning_plots/criterion_comparison_max_depth={idx[0]}_criterion={idx[1]}_n_estimators={idx[2]}.png",
    )
    plt.savefig(max_features_comparison_plot_file, bbox_inches="tight")
    plt.show()

In [ ]:
for idx, group in cv_results.groupby(
    by=["param_max_features", "param_criterion", "param_n_estimators"]
):
    fig, ax = plt.subplots()
    max_depth = ["4", "6", "8"]
    x_pos = range(len(max_depth))
    mean_4 = group.loc[group["param_max_depth"] == 4]["mean_test_score"].values[0]
    mean_6 = group.loc[group["param_max_depth"] == 6]["mean_test_score"].values[0]
    mean_8 = group.loc[group["param_max_depth"] == 8]["mean_test_score"].values[0]
    std_4 = group.loc[group["param_max_depth"] == 4]["std_test_score"].values[0]
    std_6 = group.loc[group["param_max_depth"] == 6]["std_test_score"].values[0]
    std_8 = group.loc[group["param_max_depth"] == 8]["std_test_score"].values[0]

    test_scores = [mean_4, mean_6, mean_8]
    error = [std_4, std_6, std_8]
    ax.bar(
        x_pos,
        test_scores,
        yerr=error,
        align="center",
        alpha=0.5,
        ecolor="black",
        capsize=10,
    )
    ax.set_ylabel("Test score")
    ax.set_xlabel("max_depth")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(max_depth)
    ax.set_title(
        f"max_depth comparison - param_max_features={idx[0]}, criterion={idx[1]}, n_estimators={idx[2]}"
    )
    ax.yaxis.grid(True)
    if idx[0] == "auto":
        ax.set_ylim(ymin=0.7)

    # Save the figure and show
    plt.tight_layout()
    max_depth_comparison_plot_file = os.path.join(
        ROOT_DIR,
        f"output/machine_learning_plots/criterion_comparison_param_max_features={idx[0]}_criterion={idx[1]}_n_estimators={idx[2]}.png",
    )
    plt.savefig(max_depth_comparison_plot_file, bbox_inches="tight")
    plt.show()

In [ ]:
regressor2 = RandomForestRegressor(random_state=5)
params = {
    "n_estimators": [100],
    "max_features": ["auto"],
    "max_depth": [4, 6, 8, 10, 12, 14, None],
    "criterion": ["squared_error"],
}
CV_regressor2 = GridSearchCV(estimator=regressor2, param_grid=params, cv=3, verbose=2)
CV_regressor2.fit(X_train, y_train)

In [ ]:
cv_results2 = pd.DataFrame(data=CV_regressor2.cv_results_)
cv_outfile2 = os.path.join(ROOT_DIR, "output/random-forest_search for max depth.csv")
cv_results2.to_csv(cv_outfile2, index=False)
display(pd.DataFrame(data=CV_regressor2.cv_results_))

In [ ]:
fig, ax = plt.subplots()
max_depths = [4, 6, 8, 10, 12, 14, "None"]
x_pos = range(len(max_depths))
test_scores = cv_results2["mean_test_score"]
error = cv_results2["std_test_score"]
ax.bar(
    x_pos,
    test_scores,
    yerr=error,
    align="center",
    alpha=0.5,
    ecolor="black",
    capsize=10,
)
ax.set_ylabel("Test score")
ax.set_xlabel("max_depth")
ax.set_xticks(x_pos)
ax.set_xticklabels(max_depths)
ax.set_title(f"max_depth vs test_score")
ax.yaxis.grid(True)
ax.set_ylim(ymin=0.7)

# Save the figure and show
plt.tight_layout()
max_depth_comparison_plot_file = os.path.join(
    ROOT_DIR, f"output/machine_learning_plots/max_depth_line_plot.png"
)
plt.savefig(max_depth_comparison_plot_file, bbox_inches="tight")
plt.show()